Kaggle link : https://www.kaggle.com/datasets/maverickss26/amazon-fine-food-reviews
This is a multi class classification problem but in below code I have considered it as binary class classification problem

In [1]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import contractions
from num2words import num2words
from textblob import TextBlob
from spellchecker import SpellChecker
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv("/users/amulya/Downloads/archive (6) 2/Reviews.csv")
df = df.head(10000)
df['Score'] = df['Score'].replace({5:1,4:1,3:0,2:0,1:0})
# contraction_dict = {"ain't": "are not","'s":" is","aren't": "are not", "didn't": "did not"}
df['Summary'] = df['Summary'].str.lower()
df['Summary'] = df['Summary'].apply(word_tokenize)
punctuation = set(string.punctuation)
df['Summary'] = df['Summary'].apply(lambda tokens: [token for token in tokens if token not in punctuation])
stop_words = set(stopwords.words('english'))
df['Summary'] = df['Summary'].apply(lambda tokens: [token for token in tokens if token not in stop_words])
lemmatizer = WordNetLemmatizer()
df['Summary'] = df['Summary'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
df['Summary'] = df['Summary'].apply(lambda tokens: [num2words(token) if token.isdigit() else token for token in tokens])
df['Summary'] = df['Summary'].apply(lambda tokens: [contractions.fix(token) for token in tokens])
df['Summary'] = df['Summary'].apply(lambda tokens: [token for token in tokens if token.isalnum()])
df['Summary'] = df['Summary'].apply(lambda tokens: [token.strip() for token in tokens])
# spell = SpellChecker()
# df['Summary'] = df['Summary'].apply(lambda tokens: [spell.correction(token) for token in tokens])
spell = SpellChecker()
for tokens in df['Summary']:
    for token in tokens:
        if token is None:
           pass
        else:
            corrected_token = spell.correction(token)
           
# Join tokens back to strings
df['Summary'] = df['Summary'].apply(' '.join)

# print(df)
w2v_model = Word2Vec(sentences=df['Summary'], vector_size=100, window=5, min_count=1, workers=4)
w2v_model.train(df['Summary'], total_examples=len(df['Summary']), epochs=10)
word_vectors = w2v_model.wv
print(word_vectors)
X = []
for tokens in df['Summary']:
    vector_sum = np.zeros(100)  # Initialize a zero vector of appropriate dimensionality
    for token in tokens:
        if token in word_vectors:
            vector_sum += word_vectors[token]  # Add word vector to the sum
    X.append(vector_sum)
z = pd.DataFrame(X)
y = df['Score']
X_train, X_test, y_train, y_test = train_test_split(z, y, test_size=0.2, random_state=42)
logistic_model = LogisticRegression(C= 1, fit_intercept= True, max_iter= 100, penalty= 'l2', solver= 'lbfgs')
logistic_model.fit(X_train, y_train)
y_pred_train = logistic_model.predict(X_train)
y_pred_test = logistic_model.predict(X_test)

# Evaluate the model
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


[nltk_data] Downloading package punkt to /Users/amulya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amulya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/amulya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


KeyedVectors<vector_size=100, 38 keys>
Train Accuracy: 0.762625
Test Accuracy: 0.7545


/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train Accuracy: 0.7625
Test Accuracy: 0.754

In [3]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],  # Regularization parameter
    'penalty': ['l1', 'l2'],  # Regularization penalty
    'solver': ['liblinear','lbfgs'],  # Algorithm to use in the optimization problem
    'max_iter': [10,30,50,60,100, 200, 300,1000,400,800],  # Maximum number of iterations
    'fit_intercept': [True,False]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, scoring='accuracy', cv=5)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(z, y, test_size=0.3, random_state=42)

# Fit the GridSearchCV object
grid_search.fit(x_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Evaluate the model with the best hyperparameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}')

/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or sca

Best Hyperparameters: {'C': 0.2, 'fit_intercept': True, 'max_iter': 50, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy: 0.7606666666666667
Precision: 0.7632996632996633
Recall: 0.9934268185801928
F1 Score: 0.8632901751713633


/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
1200 fits failed out of a total of 4800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1200 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/